# Peptide MD

In [1]:
import molsysmt as msm
import openmm as omm
import openmm.app as app
from openmm import unit
from mdtraj.reporters import HDF5Reporter
from sys import stdout

In [2]:
pdb = app.PDBFile('hepta-L-Arg_solv.pdb')

In [3]:
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3p.xml')

In [4]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME,
        nonbondedCutoff=1.2*unit.nanometer, constraints=app.HBonds)

In [5]:
integrator = omm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond, 0.002*unit.picoseconds)

In [6]:
platform = omm.Platform.getPlatformByName('CUDA')

In [7]:
simulation = app.Simulation(pdb.topology, system, integrator, platform)

In [8]:
simulation.context.setPositions(pdb.positions)

In [9]:
state_pre_minimization = simulation.context.getState(getEnergy=True)

In [10]:
state_pre_minimization.getPotentialEnergy()

Quantity(value=5265418125.74374, unit=kilojoule/mole)

In [11]:
simulation.minimizeEnergy()

In [12]:
state_post_minimization = simulation.context.getState(getEnergy=True)

In [13]:
state_post_minimization.getPotentialEnergy()

Quantity(value=-85126.31875983975, unit=kilojoule/mole)

In [14]:
h5_reporter = HDF5Reporter('traj_2.h5', 2500)

In [15]:
stdout_reporter = app.StateDataReporter(stdout, 1000000, step=True,
        potentialEnergy=True, temperature=True)

In [16]:
simulation.reporters.append(h5_reporter)

In [17]:
simulation.reporters.append(stdout_reporter)

In [18]:
%time simulation.step(260000000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000000,-69492.83438483975,296.61614300624655
2000000,-69235.63125983975,306.51255578407984
3000000,-69424.25625983975,308.93982334257856
4000000,-69641.03750983975,295.39029950492875
5000000,-69462.14688483975,297.4765172573027
6000000,-69343.44375983975,309.24255098799944
7000000,-69209.39688483975,292.10962848295367
8000000,-69368.25625983975,299.0367975480891
9000000,-69493.22500983975,297.7421124075511
10000000,-69525.85000983975,297.7713579113023
11000000,-69383.64688483975,301.8832953971965
12000000,-69693.17813483975,297.61672368832706
13000000,-69293.49063483975,299.95683895589923
14000000,-69430.60000983975,302.442518161023
15000000,-69249.44375983975,295.29675517391837
16000000,-69111.31875983975,299.59120869172756
17000000,-69549.47500983975,289.99987062606994
18000000,-69508.74063483975,297.6445154295964
19000000,-69299.53750983975,300.8514802964304
20000000,-69203.00625983975,295.2794952810267
21000000,-69660.25625983

In [19]:
h5_reporter.close()